In [2]:
pip install PyGithub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.9 MB/s eta 0:00:00


In [3]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00


In [4]:
from github import Github, GithubException
import nbformat
import re
import requests
import os
import openai

In [5]:
github_access_token = '<Your Github api token here>'
chatGPT_access_token = '<Your ChatGPT api token here>'
openai.api_key = chatGPT_access_token

In [49]:
client = Github(github_access_token)
username = 'aayush4396'
user_url = f'https://github.com/{username}'
user_id = user_url.split('/')[-1]
user = client.get_user(user_id)

Get all repo folders, files, content path containing only ipynb and py files in a repo

In [50]:
def get_files(contents):
  files = []
  while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
      contents.extend(repo.get_contents(file_content.path))
    else:
      file_type = file_content.name.split('.')[-1]
      if file_type=='ipynb' or file_type=='py':
        files.append(file_content)
  return files

In [51]:
repo_dict = {}
for repo in user.get_repos():
  try:
    contents = repo.get_contents('')
  except GithubException:
    print('Empty Repo')
  file_content = get_files(contents)
  if len(file_content)!=0:
    repo_dict[repo.name] = file_content

No. of Repos

In [52]:
len(repo_dict.keys())

3

1. format_nb() Function to clean a notebook if ipynb.
2. clean_code() Function to clean the code, removing import, comments, etc

In [53]:
def format_nb(notebook_content):
  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=nbformat.NO_CONVERT)
  code_cells = []
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          code_cells.append(cell.source)

  return code_cells

def clean_code(code):
  x = '\n'.join(code)
  x = re.sub(r'import.*?(\n|$)','',x)
  x = re.sub(r'#.*?(\n|$)','',x)
  x = re.sub(r'\'\'\'.*?(\n|$)','',x)
  x = re.sub(r'from.*?(\n|$)','',x)
  x = re.sub(r'!pip.*?(\n|$)','',x)
  x = re.sub(r'pd.*?(\n|$)','',x)
  x = re.sub(r'drive.*?(\n|$)','',x)
  x = re.sub(r'drive.*?(\n|$)','',x)
  x = re.sub(r'!gdown.*?(\n|$)','',x)
  x = re.sub(r'%mat.*?(\n|$)','',x)
  x = re.sub(r'warn.*?(\n|$)','',x)
  x = re.sub('\n','',x)
  # x = re.sub(' ','',x)
  return x

code_clean_url() Function to clean from a list of codes of a single repo

In [54]:
#returns the a list of codes inside the folder
def code_clean_url(contents):
  codes = []
  for content in contents:
    file_type = content.name.split('.')[-1]
    if file_type == 'ipynb':

      url_json = requests.get(content.download_url)
      text = url_json.text
      code = format_nb(text)
      code_cleaned = clean_code(code)
    else:
      url_json = requests.get(content.download_url)
      text = url_json.text
      code_cleaned = clean_code(text)
    codes.append(code_cleaned)

  return codes


Dictionary consisting of repo names as keys and all the ipynb/py codes as values

In [55]:
codes_dict = {}
for repo_name, contents in repo_dict.items():
  codes_dict[repo_name] = code_clean_url(contents)

various prompts to get the score and reason

In [56]:
first_part = 'Please retain the context of a half code that I\'m sharing with you. your next task will be dependant on this half:'
no_part = "Based on the code provided to you,rate the whole code on a scale of 1 to 100 where 1 is least complex and 100 is the most complex. Just give a numerical score, no text stricty (less than 10 tokens)"
no_reply = 'DON\'T REPLY ANYTHING ON THIS PART, no words please'
next_part = 'the next part of the code is given here:'
score_prompt = 'Based on this half and the half that i provided you, rate the whole code on a scale of 1 to 100 where 1 is least complex and 100 is the most complex. Just give a numerical score, no text stricty (less than 10 tokens)'
justify = 'Justify your rating in points and the number of tokens should be less than 100'

In [59]:
x=0
score=0
scores = []
final_scores = {}
reasons = []
final_reason = {}
#for loop for going through all the repository data stored in codes_dict
for repo_name,codes in codes_dict.items():
  x+=1
  num_chars = 3500
  #Going through one repo's all the files containing ipynb and py files
  for code in codes:
    messages = [
    {'role':'system', 'content':'You are an expert coder who checks the complexity of the code and returns an overall complexity score'}
    ]
    #If number of characters of a code is less than 3500, then below prompt will be used.
    if len(code)<num_chars:
      # print(len(code))
      messages.append({'role':'user','content':no_part+'\n'+code})
      completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=messages,
          max_tokens=200,
          stop=None,
          temperature=0.1,
          )
      score = completion.choices[0].message.content

    #If the number of characters of a code is greater than 3500, then below prompt will be used.
    else:
      # print(len(code))
      parts = len(code)//num_chars
      # print(parts)
      for i in range(parts):
        messages.append({'role':'user','content':first_part+'\n'+code[i*num_chars:(i+1)*num_chars]+no_reply})
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=200,
            stop=None,
            temperature=0.1,
            )
        messages.append({'role':'user','content':next_part})

        messages.append({'role':'user','content':score_prompt})
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=200,
            stop=None,
            temperature=0.1,
            )


      # print(score)
    score = completion.choices[0].message.content
    score = re.findall('[0-9]+',score)
    try:
      score = float(score[0])
    except:
      score = 0
    # messages = [
    #     {'role':'system', 'content':'you have to justify your answer in less than 100 token of why you gave that score.'},
    #     {'role':'user', 'content':justify}
    #     ]
    messages.append({'role':'user','content':justify})
    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=200,
            stop=None,
            temperature=0.1,
            )
    reason = completion.choices[0].message.content
    reasons.append(reason)
    scores.append(score)

  final_reason[repo_name] = '\n'.join(reasons)
  final_scores[repo_name] = sum(scores)/len(scores)
  # if x==2:
  #   break
if x==0:
  print('no repo with py or ipynb file')

You can face the issue of server overload. At the moment you can only check the usernames who have lesser number of repos.

In [60]:
final_scores

{'Object-Classification': 85.0,
 'Telecom-Churn-Analysis': 87.5,
 'Twitter-Sentiment-Analysis': 81.66666666666667}

In [61]:
for k,v in final_scores.items():
  if v==max(final_scores.values()):
    print(f'Most complex repo is {k} with a score of {round(final_scores[k],2)}/100 because {final_reason[k]}')

Most complex repo is Telecom-Churn-Analysis with a score of 87.5/100 because Overall complexity score: 75

Justification:
- The code involves loading and preprocessing image data using NumPy and TensorFlow libraries.
- It includes building and training multiple neural network models with varying complexity levels.
- The code also uses K-Fold cross-validation for model evaluation.
- There are multiple hyperparameters that can be tuned to optimize the model performance.
- The code involves plotting and visualizing the training accuracy of the models.
- The code is well-structured and organized, with clear comments and variable names.
The code seems to be moderately complex as it involves data preprocessing, model building, and evaluation. The code includes data cleaning, feature engineering, scaling, logistic regression, confusion matrix, ROC curve, and cutoff analysis. The code also involves plotting graphs for visualization. Overall, I would rate the code around 70 out of 100 in terms 